# Two cents on W&B Tables

W&B Tables accelerate the ML development lifecycle by giving users the ability to rapidly extract meaningful insights from data. The W&B Table Visualizer provides an interactive interface to perform powerful analytics functions like grouping, joining, and creating custom fields while simultaneously supporting rich media annotations such as bounding boxes and segmentation masks. 

WB Tables is designed generically to work well for a wide range of use cases - from analyzing intermediate data transformations to reviewing model predictions - while being directly integrated directly into the WB UI dashboard, allowing users to learn, adapt, and improve their models effectively and efficiently.

* Check out the official documentation [here](https://docs.wandb.ai/guides/data-vis)

* Learn more about W&B Tables from this [kernel](https://wandb.me/better-tables).


# Imports and Setup

In [ ]:
# Install W&B for visualizing dataset intereactively
!pip install -q --upgrade wandb

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
import tensorflow as tf
from PIL import Image

import wandb
wandb.login()

# Prepare Dataset

In [ ]:
train_df = pd.read_csv('../input/g2net-constant-q-transformed/training_labels.csv')
train_df.head()

In [ ]:
tmp_df = train_df.sample(frac=0.01)
tmp_df = tmp_df.reset_index(drop=True)

print(f'Size of sampled dataset: {len(tmp_df)}')

In [ ]:
plt.figure(figsize=(16, 5))
ax = sns.countplot(data=tmp_df, y="target")
ax.set_title("Distribution of Labels");
print(tmp_df.target.value_counts())

# Log Dataset as W&B Tables

In [ ]:
run = wandb.init(project='g2net-cqt-visualize', job_type='cqt-viz')

preview_at = wandb.Table(columns=['id', 'cqt', 'ground-truth'])

for i in tqdm(range(len(tmp_df))):
    row = tmp_df.loc[i]
    im = Image.open(f'../input/g2net-constant-q-transformed/train_cqt/train_cqt/{row.id}.png')
    preview_at.add_data(row.id,
                        wandb.Image(np.expand_dims(np.array(im), -1)),
                        row.target)
    
wandb.log({'g2net_cqt_viz': preview_at})
wandb.finish()

![img](https://i.imgur.com/329cY0c.gif)